# DSAA 5002 - Data Mining and Knowledge Discovery in Data Science
---

# Task 2 (50 marks) Application of Knowledge Graph

**Background:** 
**In addition to explicitly mentioning listed companies, each news article may also implicitly impact the other 
companies, either positively or negatively.**

# Q4 Knowledge-Driven Financial Analysis
---

## 1. Generate related company impact domain - a table

In [8]:
import pandas as pd
# Step 1: Read relevant tables
# Read node information table
nodes_df = pd.read_csv(r'KnowledgeGraph\hidy.nodes.company.csv')

# Read relationship information tables
compete_df = pd.read_csv(r'KnowledgeGraph\hidy.relationships.compete.csv')
cooperate_df = pd.read_csv(r'KnowledgeGraph\hidy.relationships.cooperate.csv')
dispute_df = pd.read_csv(r'KnowledgeGraph\hidy.relationships.dispute.csv')
invest_df = pd.read_csv(r'KnowledgeGraph\hidy.relationships.invest.csv')
same_industry_df = pd.read_csv(r'KnowledgeGraph\hidy.relationships.same_industry.csv')
supply_df = pd.read_csv(r'KnowledgeGraph\hidy.relationships.supply.csv')


In [23]:
from tqdm import tqdm  # Importing the tqdm module

company_relation = nodes_df.copy()

# Step 2: Iterate through each row of the node information table
for index, row in tqdm(company_relation.iterrows(), total=len(company_relation), desc='Processing'):
    company_id = row[':ID']  # Fetching the ID of the company
    
    same_impact_company_ID_set = set()
    opposite_impact_company_ID_set = set()
    
    # Searching for IDs of other companies related to this company in the relationship tables
    # Please check and add to the respective sets based on each type of relationship
    
    # Searching for related company IDs in 'hidy.relationships.invest.csv' and 'hidy.relationships.supply.csv' tables
    invest_related = invest_df[invest_df[':START_ID'] == company_id]
    supply_related = supply_df[supply_df[':START_ID'] == company_id]
    # Adding ':END_ID' values of related rows to the sets
    same_impact_company_ID_set.update(set(invest_related[':END_ID'].values))
    same_impact_company_ID_set.update(set(supply_related[':END_ID'].values))

    # Searching for related company IDs in 'hidy.relationships.cooperate.csv' and 'hidy.relationships.same_industry.csv' tables
    cooperate_related = cooperate_df[(cooperate_df[':START_ID'] == company_id) | (cooperate_df[':END_ID'] == company_id)]
    same_industry_related = same_industry_df[(same_industry_df[':START_ID'] == company_id) | (same_industry_df[':END_ID'] == company_id)]
    # Adding ':START_ID' and ':END_ID' values of related rows to the sets
    same_impact_company_ID_set.update(set(cooperate_related[':START_ID'].values))
    same_impact_company_ID_set.update(set(cooperate_related[':END_ID'].values))
    same_impact_company_ID_set.update(set(same_industry_related[':START_ID'].values))
    same_impact_company_ID_set.update(set(same_industry_related[':END_ID'].values))

    # Searching for related company IDs in 'hidy.relationships.compete.csv' and 'hidy.relationships.dispute.csv' tables
    compete_related = compete_df[(compete_df[':START_ID'] == company_id) | (compete_df[':END_ID'] == company_id)]
    dispute_related = dispute_df[(dispute_df[':START_ID'] == company_id) | (dispute_df[':END_ID'] == company_id)]
    # Adding ':START_ID' and ':END_ID' values of related rows to the sets
    opposite_impact_company_ID_set.update(set(compete_related[':START_ID'].values))
    opposite_impact_company_ID_set.update(set(compete_related[':END_ID'].values))
    opposite_impact_company_ID_set.update(set(dispute_related[':START_ID'].values))
    opposite_impact_company_ID_set.update(set(dispute_related[':END_ID'].values))
    

    # Creating two lists to store the company names that have same and opposite impacts
    same_impact_company_list = []
    opposite_impact_company_list = []

    # Iterating through the related IDs and fetching the corresponding company names from the node information table, adding them to the lists
    for company_id in same_impact_company_ID_set:
        company_name = nodes_df[nodes_df[':ID'] == company_id]['company_name'].values
        if len(company_name) > 0:
            same_impact_company_list.extend(company_name)

    for company_id in opposite_impact_company_ID_set:
        company_name = nodes_df[nodes_df[':ID'] == company_id]['company_name'].values
        if len(company_name) > 0:
            opposite_impact_company_list.extend(company_name)
    
    # Adding the obtained company name lists to the corresponding columns in the 'company_relation' DataFrame
    if len(same_impact_company_list)  != 0:
        company_relation.at[index,'same_impact_company'] = ", ".join(same_impact_company_list)
    if len(opposite_impact_company_list)  != 0:
        company_relation.at[index,'opposite_impact_company'] = ", ".join(opposite_impact_company_list)
    
# Storing the results into an Excel file
company_relation.to_excel(r'KnowledgeGraph\company_relationship.xlsx', index=False)


Processing: 100%|█████████████████████████████████████████████████████████████████| 3974/3974 [00:17<00:00, 228.93it/s]


In [24]:
company_relation

,:ID,company_name,code,:LABEL,same_impact_company,opposite_impact_company
0,0,东诚药业,002675.SZ,company,"东诚药业, 同仁堂, 海翔药业, 福安药业",NaN
1,1,大庆华科,000985.SZ,company,NaN,NaN
2,2,恒辉安防,300952.SZ,company,"朗科智能, 恒辉安防, 彤程新材",NaN
3,3,康跃科技,300391.SZ,company,"以岭药业, 康跃科技","恒通科技, 康跃科技"
4,4,诚益通,300430.SZ,company,"大湖股份, 诚益通",NaN
...,...,...,...,...,...,...
3969,3969,圣诺生物,688117.SH,company,"圣诺生物, 上声电子",NaN
3970,3970,牧原股份,002714.SZ,company,"新希望, 牧原股份, 海螺水泥, 光大银行, 中牧股份, 智动力, 温氏股份, 立华股份, ...","牧原股份, 招商证券"
3971,3971,中航高科,600862.SH,company,NaN,NaN
3972,3972,江丰电子,300666.SZ,company,"北京君正, 紫光国微, 江丰电子, 秦川机床, 雅克科技, 亿通科技, 贵研铂业, 沪硅产业...",NaN


## 2. Generate the Knowledge-Driven Financial Analysis result use the table above

In [12]:
company_relation = pd.read_excel(r'KnowledgeGraph\company_relationship.xlsx')

In [14]:
# Reading Application_set.xlsx
source_file = r'Result_dataset\\Application_set_BiLSTM\\Application_set_labeled_with_BiLSTM_v2.xlsx'
df = pd.read_excel(source_file)

# Adding two new columns "Implicit_Positive_Company" and "Implicit_Negative_Company" in Analysis_set.xlsx
df['Implicit_Positive_Company'] = ""
df['Implicit_Negative_Company'] = ""

In [56]:
from tqdm import tqdm  # Importing tqdm module

# Iterating over each row in Application_set.xlsx
for index, row in tqdm(df.iterrows(), total=len(df), desc='Processing'):
    explicit_companies = row['Explicit_Company'].split(', ')
    
    # Step 1: Creating two sets for same impact companies and opposite impact companies
    same_impact_company_set = set()
    opposite_impact_company_set = set()
    
    # Step 2: Iterating over each company in Explicit_Company
    for company in explicit_companies:
        # Finding company ID
        if_company_id_in_companies_name = company_relation[nodes_df['company_name'] == company][':ID'].values
        if len(if_company_id_in_companies_name) == 0:
            continue
        company_id = if_company_id_in_companies_name[0]
        
        same_impact_company_series = company_relation.loc[company_relation[':ID'] == company_id]["same_impact_company"]
        same_impact_company_of_this_company = same_impact_company_series.apply(lambda x: x.split(', ') if isinstance(x, str) else [])
        if len(same_impact_company_of_this_company.iloc[0]) != 0:
            same_impact_company_set.update(set(str(x) for x in same_impact_company_of_this_company.explode()))

        opposite_impact_company_series = company_relation.loc[company_relation[':ID'] == company_id]["opposite_impact_company"]
        opposite_impact_company_of_this_company = opposite_impact_company_series.apply(lambda x: x.split(', ') if isinstance(x, str) else [])
        if len(opposite_impact_company_of_this_company.iloc[0]) != 0:        
            opposite_impact_company_set.update(set(str(x) for x in opposite_impact_company_of_this_company.explode()))

    # Step 3: Populating based on label values
    if row['label'] == 1:
        df.at[index, 'Implicit_Positive_Company'] = ', '.join(same_impact_company_set) if same_impact_company_set else 'None'
        df.at[index, 'Implicit_Negative_Company'] = ', '.join(opposite_impact_company_set) if opposite_impact_company_set else 'None'
    elif row['label'] == 0:
        df.at[index, 'Implicit_Positive_Company'] = ', '.join(opposite_impact_company_set) if opposite_impact_company_set else 'None'
        df.at[index, 'Implicit_Negative_Company'] = ', '.join(same_impact_company_set) if same_impact_company_set else 'None'

Processing: 100%|█████████████████████████████████████████████████████████████| 524226/524226 [18:50<00:00, 463.77it/s]


In [57]:
df

,NewsID,NewsContent,Explicit_Company,label,Implicit_Positive_Company,Implicit_Negative_Company
0,1,建设银行原董事长张恩照一审被判15年 本报记者 田雨 李京华 中国建设银行股份...,建设银行,0,"任子行, 捷捷微电, 建设银行","上海银行, 中国人寿, 中金公司, 民生银行, 怡亚通, 兴业银行, 平安银行, 贵州茅台,..."
1,2,农行信用卡中心搬到上海滩 中国农业银行信用卡中心由北京搬到上海了！ 农行行长杨明生...,农业银行,1,"上海银行, 中国国航, TCL集团, 中国神华, 兴业银行, 山东矿机, 申万宏源, 中信证...","ST云维, 农业银行"
2,3,外运发展：价值型蓝筹股补涨要求强烈 在新基金快速发行以及申购资金回流的情况下，市场总体上...,"中国国航, 外运发展",1,"中国国航, 中国石化, 南方航空, 寒武纪, 中国外运, 中国电建, 吉祥航空, 山航B, ...",None
3,4,胜利股份：稳步走强形成标准上升通道 胜利股份（000407）公司子公司填海造地2800亩...,胜利股份,1,"新疆浩源, 特锐德, 胜利股份",None
4,5,[港股快讯]恒指收市报18960点 成交467亿港元 全景网11月30日讯 外围股市造好...,新世界股份,1,None,None
...,...,...,...,...,...,...
524221,1037031,亿华通：公司电解槽相关产品目前还处于产品的研发及测试阶段 尚未实现批量销售 每经AI快讯，有...,亿华通,1,"仕佳光子, 东风汽车, 中国船舶, 亿华通, 福田汽车, 百奥泰, 宝泰隆",None
524222,1037032,依米康：接受中泰证券调研 依米康（SZ 300249，收盘价：10.38元）发布公告称，20...,"中泰证券, 依米康",1,"龙磁科技, 乐歌股份, 中金公司, 国新能源, 中国平安, 东方证券, 申万宏源, 中信证券...","西水股份, 华谊嘉信, 中泰证券"
524223,1037033,天风证券给予中核科技买入评级 核电行业景气上行 公司有望乘风而起 天风证券10月13日发布研...,"天风证券, 中核科技",1,"西南证券, 久远银海, 凯撒文化, 国信证券, 中金公司, 闻泰科技, 华泰证券, 上海临港...","三特索道, 天风证券, 吉翔股份, 中源家居"
524224,1037034,海特生物：公司在抗癌药CPT获批后 会考虑适时开展CPT在海外的临床并谋求上市 有投资者提问...,海特生物,1,"药明康德, 海特生物, 海尔生物",None


In [ ]:
# Analysis_set.xlsx is our Task2.xlsx
destination_file = r'Result_dataset\\Analysis_set_BiLSTM\\Analysis_set_labeled_with_BiLSTM_v1.xlsx'
df.to_excel(destination_file, index=False)

**Analysis_set_labeled_with_BiLSTM_v1.xlsx is our Task2.xlsx**


In [ ]:
# Analysis_set_labeled_with_BiLSTM_v1.xlsx is our Task2.xlsx
destination_file = 'Task2.xlsx'
df_application.to_excel(destination_file, index=False)

---

## 2. Slower Method

In [54]:
# from tqdm import tqdm  # 导入tqdm模块

# # Step 3: 遍历Application_set.xlsx表格的每一行
# for index, row in tqdm(df.iterrows(), total=len(df), desc='Processing'):
#     explicit_companies = row['Explicit_Company'].split(', ')
    
#     # Step 3.1: 创建两个集合，构造相同影响的公司域和相反影响的公司域
#     same_impact_company_ID_set = set()
#     opposite_impact_company_ID_set = set()
    
#     # Step 3.2: 遍历Explicit_Company中的每一个公司
#     for company in explicit_companies:
#         # 查找公司ID
#         if_company_id_in_companies_name = nodes_df[nodes_df['company_name'] == company][':ID'].values
#         if not if_company_id_in_companies_name:#若没有找到公司ID（为空），则跳过
#             continue
#         company_id = if_company_id_in_companies_name[0]
        
#         # 在hidy.relationships.invest.csv和hidy.relationships.supply.csv两个单向关系的表格中搜索相关公司ID
#         invest_related = invest_df[invest_df[':START_ID'] == company_id]
#         supply_related = supply_df[supply_df[':START_ID'] == company_id]
#         # 将相关行的':END_ID'都放入集合
#         same_impact_company_ID_set.update(set(invest_related[':END_ID'].values))
#         same_impact_company_ID_set.update(set(supply_related[':END_ID'].values))

#         # 在hidy.relationships.cooperate.csv和hidy.relationships.same_industry.csv两个双向关系的表格中搜索相关公司ID对应的行
#         cooperate_related = cooperate_df[(cooperate_df[':START_ID'] == company_id) | (cooperate_df[':END_ID'] == company_id)]
#         same_industry_related = same_industry_df[(same_industry_df[':START_ID'] == company_id) | (same_industry_df[':END_ID'] == company_id)]
#         # 将相关行的':START_ID'和':END_ID'都放入集合
#         same_impact_company_ID_set.update(set(cooperate_related[':START_ID'].values))
#         same_impact_company_ID_set.update(set(cooperate_related[':END_ID'].values))
#         same_impact_company_ID_set.update(set(same_industry_related[':START_ID'].values))
#         same_impact_company_ID_set.update(set(same_industry_related[':END_ID'].values))

#         # 在hidy.relationships.compete.csv和hidy.relationships.dispute.csv两个双向关系的表格中搜索相关公司ID对应的行
#         compete_related = compete_df[(compete_df[':START_ID'] == company_id) | (compete_df[':END_ID'] == company_id)]
#         dispute_related = dispute_df[(dispute_df[':START_ID'] == company_id) | (dispute_df[':END_ID'] == company_id)]
#         # 将相关行的':START_ID'和':END_ID'都放入集合
#         opposite_impact_company_ID_set.update(set(compete_related[':START_ID'].values))
#         opposite_impact_company_ID_set.update(set(compete_related[':END_ID'].values))
#         opposite_impact_company_ID_set.update(set(dispute_related[':START_ID'].values))
#         opposite_impact_company_ID_set.update(set(dispute_related[':END_ID'].values))

    
#     # Step 3.4: 根据公司ID获取公司的中文名并存入相应列表
#     same_impact_company_list = []
#     opposite_impact_company_list = []
    
#     for company_id in same_impact_company_ID_set:
#         company_name = nodes_df[nodes_df[':ID'] == company_id]['company_name'].values[0]
#         same_impact_company_list.append(company_name)

#     for company_id in opposite_impact_company_ID_set:
#         company_name = nodes_df[nodes_df[':ID'] == company_id]['company_name'].values[0]
#         opposite_impact_company_list.append(company_name)


#     # Step 3.5: 根据label值进行填充
#     if row['label'] == 1:
#         df.at[index, 'Implicit_Positive_Company'] = ', '.join(same_impact_company_list) if same_impact_company_list else 'None'
#         df.at[index, 'Implicit_Negative_Company'] = ', '.join(opposite_impact_company_list) if opposite_impact_company_list else 'None'
#     elif row['label'] == 0:
#         df.at[index, 'Implicit_Positive_Company'] = ', '.join(opposite_impact_company_list) if opposite_impact_company_list else 'None'
#         df.at[index, 'Implicit_Negative_Company'] = ', '.join(same_impact_company_list) if same_impact_company_list else 'None'

# # Step 4: 将结果存储到Analysis_set.xlsx
# destination_file = r'D:\\ProjectHub\\Jupyter Notebook\\DSAA 5002 DM\\DM-Project\\Result_dataset\\Analysis_set_BiLSTM\\Analysis_set_labeled_with_BiLSTM.xlsx'
# df.to_excel(destination_file, index=False)


Processing:   0%|                                                                | 1/524226 [00:00<42:31:11,  3.42it/s]C:\Users\HUAWEI\AppData\Local\Temp\ipykernel_95808\2286040654.py:15: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if not if_company_id_in_companies_name:#若没有找到公司ID（为空），则跳过
Processing:   0%|                                                               | 376/524226 [00:06<2:23:32, 60.83it/s]


KeyboardInterrupt: 